In [81]:
import pandas as pd
import numpy as np
import json
import requests
import bs4 as bs
import urllib.request

# Movies till 2016

In [2]:
# This dataset contains movies only uptill 2016
m_t_2016 = pd.read_csv('datasets/movie_metadata.csv')

In [3]:
# these features will be used for recommendation
m_t_2016 = m_t_2016.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [4]:
# replace all nas with unknown
for i in ["actor_1_name","actor_2_name","actor_3_name","director_name"]:
    m_t_2016[i] = m_t_2016[i].replace(np.nan, 'unknown')

In [5]:
# spaceing between genres
m_t_2016['genres'] = m_t_2016['genres'].str.replace('|', ' ')
# cleaning the movie titles
m_t_2016['movie_title'] = m_t_2016['movie_title'].str.lower()
m_t_2016['movie_title'] = m_t_2016['movie_title'].apply(lambda x : x[:-1])

"pirates of the caribbean: at world's end"

In [102]:
m_t_2016['comb'] = m_t_2016['actor_1_name'] + ' ' + m_t_2016['actor_2_name'] + ' '+ m_t_2016['actor_3_name'] + ' '+ m_t_2016['director_name'] +' ' + m_t_2016['genres']

In [107]:
m_t_2016.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...


# Movies for 2017

In [108]:
creds = pd.read_csv('datasets/credits.csv')
# get movies from 2017
mov_meta = pd.read_csv('datasets/movies_metadata.csv')

C:\Users\iamen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [109]:
mov_meta['release_date'] = pd.to_datetime(mov_meta['release_date'], errors='coerce')
mov_meta['year'] = mov_meta['release_date'].dt.year

In [110]:
new_mov_meta = mov_meta.loc[mov_meta.year == 2017,['genres','id','title','year']]
new_mov_meta['id'] = new_mov_meta['id'].astype(int)

In [111]:
m_t_2017 = pd.merge(new_mov_meta, creds, on='id')

In [112]:
# evaluates an expression node or a string containing a Python literal or container display
import ast
m_t_2017['genres'] = m_t_2017['genres'].map(lambda x: ast.literal_eval(x))
m_t_2017['cast'] = m_t_2017['cast'].map(lambda x: ast.literal_eval(x))
m_t_2017['crew'] = m_t_2017['crew'].map(lambda x: ast.literal_eval(x))

In [113]:
def make_genresList(x):
    gen = []
    st = " "
    for i in x:
        if i.get('name') == 'Science Fiction':
            scifi = 'Sci-Fi'
            gen.append(scifi)
        else:
            gen.append(i.get('name'))
    if gen == []:
        return np.NaN
    else:
        return (st.join(gen))

In [114]:
def get_actor1(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == []:
        return np.NaN
    else:
        return (casts[0])

In [115]:
def get_actor2(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=1:
        return np.NaN
    else:
        return (casts[1])

In [116]:
def get_actor3(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=2:
        return np.NaN
    else:
        return (casts[2])

In [117]:
def get_directors(x):
    dt = []
    st = " "
    for i in x:
        if i.get('job') == 'Director':
            dt.append(i.get('name'))
    if dt == []:
        return np.NaN
    else:
        return (st.join(dt))

In [118]:
m_t_2017['genres_list'] = m_t_2017['genres'].map(lambda x: make_genresList(x))
m_t_2017['actor_1_name'] = m_t_2017['cast'].map(lambda x: get_actor1(x))
m_t_2017['actor_2_name'] = m_t_2017['cast'].map(lambda x: get_actor2(x))
m_t_2017['actor_3_name'] = m_t_2017['cast'].map(lambda x: get_actor3(x))
m_t_2017['director_name'] = m_t_2017['crew'].map(lambda x: get_directors(x))

In [119]:
m_t_2017 = m_t_2017.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres_list','title']]

In [120]:
m_t_2017 = m_t_2017.dropna(how='any')

In [121]:
m_t_2017 = m_t_2017.rename(columns={'genres_list':'genres'})
m_t_2017 = m_t_2017.rename(columns={'title':'movie_title'})
m_t_2017['movie_title'] = m_t_2017['movie_title'].str.lower()
m_t_2017['comb'] = m_t_2017['actor_1_name'] + ' ' + m_t_2017['actor_2_name'] + ' '+ m_t_2017['actor_3_name'] + ' '+ m_t_2017['director_name'] +' ' + m_t_2017['genres']

In [122]:
m_t_2017.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Joachim Rønning Espen Sandberg,Johnny Depp,Javier Bardem,Geoffrey Rush,Adventure Action Fantasy Comedy,pirates of the caribbean: dead men tell no tales,Johnny Depp Javier Bardem Geoffrey Rush Joachi...
1,Zack Snyder,Ben Affleck,Henry Cavill,Gal Gadot,Action Adventure Fantasy Sci-Fi,justice league,Ben Affleck Henry Cavill Gal Gadot Zack Snyder...
2,Taika Waititi,Chris Hemsworth,Tom Hiddleston,Cate Blanchett,Action Adventure Fantasy Sci-Fi,thor: ragnarok,Chris Hemsworth Tom Hiddleston Cate Blanchett ...
3,James Gunn,Chris Pratt,Zoe Saldana,Dave Bautista,Action Adventure Comedy Sci-Fi,guardians of the galaxy vol. 2,Chris Pratt Zoe Saldana Dave Bautista James Gu...
4,Sean McNamara,Pierce Brosnan,William Hurt,Benjamin Walker,Fantasy Action Adventure,the king's daughter,Pierce Brosnan William Hurt Benjamin Walker Se...


# MOVIES FOR 2018

In [56]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
lr = {}
for i in range(4):
    lr[i] = pd.read_html(link, header=0)[i+2]
m_t_2018 = lr[0].append(lr[1].append(lr[2].append(lr[3],ignore_index=True),ignore_index=True),ignore_index=True)

In [57]:
# pip install tmdbv3api
from tmdbv3api import TMDb
tmdb = TMDb()
tmdb.api_key = '6k613257d26ba56689lg25521f9d4886'

In [58]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [59]:
m_t_2018['genres'] = m_t_2018['Title'].map(lambda x: get_genre(str(x)))

In [60]:
m_t_2018 = m_t_2018[['Title','Cast and crew','genres']]

In [62]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [64]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [65]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [66]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [67]:
m_t_2018['director_name'] = m_t_2018['Cast and crew'].map(lambda x: get_director(x))
m_t_2018['actor_1_name'] = m_t_2018['Cast and crew'].map(lambda x: get_actor1(x))
m_t_2018['actor_2_name'] = m_t_2018['Cast and crew'].map(lambda x: get_actor2(x))
m_t_2018['actor_3_name'] = m_t_2018['Cast and crew'].map(lambda x: get_actor3(x))

In [68]:
m_t_2018 = m_t_2018.rename(columns={'Title':'movie_title'})
m_t_2018 = m_t_2018.drop(['Cast and crew'], axis = 1)

In [69]:
m_t_2018.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson


In [70]:
m_t_2018['actor_2_name'] = m_t_2018['actor_2_name'].replace(np.nan, 'unknown')
m_t_2018['actor_3_name'] = m_t_2018['actor_3_name'].replace(np.nan, 'unknown')
m_t_2018['movie_title'] = m_t_2018['movie_title'].str.lower()
m_t_2018['comb'] = m_t_2018['actor_1_name'] + ' ' + m_t_2018['actor_2_name'] + ' '+ m_t_2018['actor_3_name'] + ' '+ m_t_2018['director_name'] +' ' + m_t_2018['genres']

# MOVIES FOR 2019

In [71]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
lr = {}
for i in range(4):
    lr[i] = pd.read_html(link, header=0)[i+3]
m_t_2019 = lr[0].append(lr[1].append(lr[2].append(lr[3],ignore_index=True),ignore_index=True),ignore_index=True)

In [72]:
m_t_2019['genres'] = m_t_2019['Title'].map(lambda x: get_genre(str(x)))
m_t_2019 = m_t_2019[['Title','Cast and crew','genres']]

In [77]:
m_t_2019['director_name'] = m_t_2019['Cast and crew'].map(lambda x: get_director(str(x)))
m_t_2019['actor_1_name'] = m_t_2019['Cast and crew'].map(lambda x: get_actor1(x))
m_t_2019['actor_2_name'] = m_t_2019['Cast and crew'].map(lambda x: get_actor2(x))
m_t_2019['actor_3_name'] = m_t_2019['Cast and crew'].map(lambda x: get_actor3(x))

In [78]:
m_t_2019 = m_t_2019.rename(columns={'Title':'movie_title'})
m_t_2019 = m_t_2019.drop(['Cast and crew'], axis = 1)

In [79]:
m_t_2019['actor_2_name'] = m_t_2019['actor_2_name'].replace(np.nan, 'unknown')
m_t_2019['actor_3_name'] = m_t_2019['actor_3_name'].replace(np.nan, 'unknown')
m_t_2019['movie_title'] = m_t_2019['movie_title'].str.lower()
m_t_2019['comb'] = m_t_2019['actor_1_name'] + ' ' + m_t_2019['actor_2_name'] + ' '+ m_t_2019['actor_3_name'] + ' '+ m_t_2019['director_name'] +' ' + m_t_2019['genres']

In [80]:
m_t_2019.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,escape room,Horror Thriller,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll,Taylor Russell Logan Miller Deborah Ann Woll A...
1,rust creek,Thriller Drama,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan,Hermione Corfield Jay Paulson Sean O'Bryan Jen...
2,american hangman,Thriller,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Donald Sutherland Vincent Kartheiser Oliver De...
3,a dog's way home,Drama Adventure Family,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Bryce Dallas Howard Edward James Olmos Alexand...
4,the upside,Comedy Drama,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman,Bryan Cranston Kevin Hart Nicole Kidman Neil B...


# MOVIES 2020

In [82]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')
tables = soup.find_all('table',class_='wikitable sortable')

In [84]:
lr = {}
for i in range(3):
    lr[i] =  pd.read_html(str(tables[i]))[0]
lr[3] = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'
m_t_2020 = lr[0].append(lr[1].append(lr[2].append(lr[3],ignore_index=True),ignore_index=True),ignore_index=True)

In [87]:
m_t_2020 = m_t_2020[['Title','Cast and crew']]


In [89]:
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [90]:
m_t_2020['genres'] = m_t_2020['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-90-b2d977e5f8f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_t_2020['genres'] = m_t_2020['Title'].map(lambda x: get_genre(str(x)))


In [92]:
m_t_2020['director_name'] = m_t_2020['Cast and crew'].map(lambda x: get_director(str(x)))
m_t_2020['actor_1_name'] = m_t_2020['Cast and crew'].map(lambda x: get_actor1(x))
m_t_2020['actor_2_name'] = m_t_2020['Cast and crew'].map(lambda x: get_actor2(x))
m_t_2020['actor_3_name'] = m_t_2020['Cast and crew'].map(lambda x: get_actor3(x))

<ipython-input-92-f5010f913663>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_t_2020['director_name'] = m_t_2020['Cast and crew'].map(lambda x: get_director(str(x)))
<ipython-input-92-f5010f913663>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_t_2020['actor_1_name'] = m_t_2020['Cast and crew'].map(lambda x: get_actor1(x))


In [93]:
m_t_2020 = m_t_2020.rename(columns={'Title':'movie_title'})
m_t_2020 = m_t_2020.drop(['Cast and crew'], axis = 1)

In [95]:
m_t_2020['comb'] = m_t_2020['actor_1_name'] + ' ' + m_t_2020['actor_2_name'] + ' '+ m_t_2020['actor_3_name'] + ' '+ m_t_2020['director_name'] +' ' + m_t_2020['genres']

In [97]:
m_t_2020 = m_t_2020.dropna(how='any')
m_t_2020['movie_title'] = m_t_2020['movie_title'].str.lower()

In [99]:
m_t_2020.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,the grudge,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Andrea Riseborough Demián Bichir John Cho Nico...
1,underwater,Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Kristen Stewart Vincent Cassel Jessica Henwick...
2,like a boss,Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,three christs,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Richard Gere Peter Dinklage Walton Goggins Jon...
4,inherit the viper,Drama Thriller Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Josh Hartnett Margarita Levieva Chandler Riggs...


In [123]:
clean_mov = m_t_2016.append(m_t_2017)
clean_mov.drop_duplicates(subset ="movie_title", keep = 'last', inplace = True)
clean_mov = clean_mov.append(m_t_2018,ignore_index=True)
clean_mov = clean_mov.append(m_t_2019,ignore_index=True)
clean_mov = clean_mov.append(m_t_2020,ignore_index=True)
clean_mov = clean_mov.dropna(how='any')
clean_mov.to_csv('datasets/main_data.csv',index=False)